In [ ]:
from cryptography.fernet import Fernet
import getpass
import json
import os
import string
import secrets

class PasswordManager:
    def __init__(self, key_file="secret.key", data_file="passwords.json", default_category="General"):
        self.key_file = key_file
        self.data_file = data_file
        self.default_category = default_category
        self.load_key()

    def load_key(self):
        if not os.path.exists(self.key_file):
            key = Fernet.generate_key()
            with open(self.key_file, "wb") as key_file:
                key_file.write(key)
        else:
            with open(self.key_file, "rb") as key_file:
                key = key_file.read()
        self.cipher_suite = Fernet(key)

    def encrypt(self, data):
        return self.cipher_suite.encrypt(data.encode())

    def decrypt(self, encrypted_data):
        return self.cipher_suite.decrypt(encrypted_data).decode()

    def load_passwords(self):
        if not os.path.exists(self.data_file):
            return {}
        with open(self.data_file, "rb") as data_file:
            encrypted_data = data_file.read()
        decrypted_data = self.decrypt(encrypted_data)
        return json.loads(decrypted_data)

    def save_passwords(self, passwords):
        encrypted_data = self.encrypt(json.dumps(passwords))
        with open(self.data_file, "wb") as data_file:
            data_file.write(encrypted_data)

    def generate_strong_password(self, length=12):
        characters = string.ascii_letters + string.digits + string.punctuation
        return ''.join(secrets.choice(characters) for _ in range(length))

    def add_password(self, username):
        passwords = self.load_passwords()
        if self.default_category not in passwords:
            passwords[self.default_category] = {}

        password = getpass.getpass("Enter the password: ")
        passwords[self.default_category][username] = password

        self.save_passwords(passwords)
        print("Password added successfully.")

    def retrieve_password(self, username):
        passwords = self.load_passwords()
        if self.default_category in passwords and username in passwords[self.default_category]:
            return passwords[self.default_category][username]
        return "Password not found."

    def update_password(self, username):
        passwords = self.load_passwords()
        if self.default_category in passwords and username in passwords[self.default_category]:
            new_password = getpass.getpass("Enter the new password: ")
            passwords[self.default_category][username] = new_password
            self.save_passwords(passwords)
            print("Password updated successfully.")
        else:
            print("Password not found. Update failed.")

    def delete_password(self, username):
        passwords = self.load_passwords()
        if self.default_category in passwords and username in passwords[self.default_category]:
            del passwords[self.default_category][username]
            self.save_passwords(passwords)
            print("Password deleted successfully.")
        else:
            print("Password not found. Deletion failed.")


if __name__ == "__main__":
    password_manager = PasswordManager()

    while True:
        print("\nPassword Manager Menu:")
        print("1. Set Password")
        print("2. View Password")
        print("3. Change Password")
        print("4. Exit")

        choice = input("Enter your choice (1-4): ")

        if choice == "1":
            username = input("Enter the username: ")
            password_manager.add_password(username)
        elif choice == "2":
            username = input("Enter the username: ")
            retrieved_password = password_manager.retrieve_password(username)
            print(f"Retrieved Password: {retrieved_password}")
        elif choice == "3":
            username = input("Enter the username: ")
            password_manager.update_password(username)
        elif choice == "4":
            print("Exiting Password Manager. Goodbye!")
            break
        else:
            print("Invalid choice. Please choose a number between 1 and 4.")
